# IBM watsonx.ai

>[WatsonxLLM](https://ibm.github.io/watsonx-ai-python-sdk/fm_extensions.html#langchain) 是 IBM [watsonx.ai](https://www.ibm.com/products/watsonx-ai) 基础模型的封装器。

此示例展示了如何使用 `LangChain` 与 `watsonx.ai` 模型进行通信。

## 概述

### 集成详情
| 类 | 包 | 本地 | 可序列化 | [JS 支持](https://js.langchain.com/docs/integrations/llms/ibm/) | 包下载量 | 最新包 |
| :--- | :--- | :---: | :---: |  :---: | :---: | :---: |
| [WatsonxLLM](https://python.langchain.com/api_reference/ibm/llms/langchain_ibm.llms.WatsonxLLM.html) | [langchain-ibm](https://python.langchain.com/api_reference/ibm/index.html) | ❌ | ❌ | ✅ | ![PyPI - Downloads](https://img.shields.io/pypi/dm/langchain-ibm?style=flat-square&label=%20) | ![PyPI - Version](https://img.shields.io/pypi/v/langchain-ibm?style=flat-square&label=%20) |

## 设置

要访问 IBM watsonx.ai 模型，您需要创建 IBM watsonx.ai 帐户、获取 API 密钥，并安装 `langchain-ibm` 集成软件包。

### 凭证

下面的单元格定义了使用 watsonx Foundation Model 推理所需的凭证。

**操作：** 提供 IBM Cloud 用户 API 密钥。有关详细信息，请参阅
[管理用户 API 密钥](https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui)。

In [2]:
import os
from getpass import getpass

watsonx_api_key = getpass()
os.environ["WATSONX_APIKEY"] = watsonx_api_key

此外，您还可以将其他 Secret 作为环境变量传递。

In [ ]:
import os

os.environ["WATSONX_URL"] = "your service instance url"
os.environ["WATSONX_TOKEN"] = "your token for accessing the CPD cluster"
os.environ["WATSONX_PASSWORD"] = "your password for accessing the CPD cluster"
os.environ["WATSONX_USERNAME"] = "your username for accessing the CPD cluster"
os.environ["WATSONX_INSTANCE_ID"] = "your instance_id for accessing the CPD cluster"

### 安装

LangChain IBM 集成位于 `langchain-ibm` 包中：

In [ ]:
!pip install -qU langchain-ibm

## 实例化

您可能需要为不同的模型或任务调整模型 `parameters`。有关详细信息，请参阅 [文档](https://ibm.github.io/watsonx-ai-python-sdk/fm_model.html#metanames.GenTextParamsMetaNames)。

In [1]:
parameters = {
    "decoding_method": "sample",
    "max_new_tokens": 100,
    "min_new_tokens": 1,
    "temperature": 0.5,
    "top_k": 50,
    "top_p": 1,
}

使用先前设置的参数初始化 `WatsonxLLM` 类。


**注意**:

- 要为 API 调用提供上下文，必须添加 `project_id` 或 `space_id`。更多信息请参阅[文档](https://www.ibm.com/docs/en/watsonx-as-a-service?topic=projects)。
- 根据您预配的服务实例的区域，使用此处描述的其中一个 URL ([documentation](https://ibm.github.io/watsonx-ai-python-sdk/setup_cloud.html#authentication))。

在此示例中，我们将使用 `project_id` 和 Dallas URL。


您需要指定将用于推理的 `model_id`。所有可用模型均可在[文档](https://ibm.github.io/watsonx-ai-python-sdk/fm_model.html#TextModels)中找到。

In [4]:
from langchain_ibm import WatsonxLLM

watsonx_llm = WatsonxLLM(
    model_id="ibm/granite-13b-instruct-v2",
    url="https://us-south.ml.cloud.ibm.com",
    project_id="PASTE YOUR PROJECT_ID HERE",
    params=parameters,
)

或者，您也可以使用 Cloud Pak for Data 凭据。有关详细信息，请参阅[文档](https://ibm.github.io/watsonx-ai-python-sdk/setup_cpd.html)。

In [ ]:
watsonx_llm = WatsonxLLM(
    model_id="ibm/granite-13b-instruct-v2",
    url="PASTE YOUR URL HERE",
    username="PASTE YOUR USERNAME HERE",
    password="PASTE YOUR PASSWORD HERE",
    instance_id="openshift",
    version="4.8",
    project_id="PASTE YOUR PROJECT_ID HERE",
    params=parameters,
)

除了 `model_id`，你也可以传入先前微调模型的 `deployment_id`。模型微调的完整流程请参阅 [Working with TuneExperiment and PromptTuner](https://ibm.github.io/watsonx-ai-python-sdk/pt_tune_experiment_run.html)。

In [ ]:
watsonx_llm = WatsonxLLM(
    deployment_id="PASTE YOUR DEPLOYMENT_ID HERE",
    url="https://us-south.ml.cloud.ibm.com",
    project_id="PASTE YOUR PROJECT_ID HERE",
    params=parameters,
)

对于某些需求，可以选择将 IBM 的 [`APIClient`](https://ibm.github.io/watsonx-ai-python-sdk/base.html#apiclient) 对象传递到 `WatsonxLLM` 类中。

In [ ]:
from ibm_watsonx_ai import APIClient

api_client = APIClient(...)

watsonx_llm = WatsonxLLM(
    model_id="ibm/granite-13b-instruct-v2",
    watsonx_client=api_client,
)

你也可以将 IBM 的 [`ModelInference`](https://ibm.github.io/watsonx-ai-python-sdk/fm_model_inference.html) 对象传递给 `WatsonxLLM` 类。

In [ ]:
from ibm_watsonx_ai.foundation_models import ModelInference

model = ModelInference(...)

watsonx_llm = WatsonxLLM(watsonx_model=model)

## 调用方式
要获取补全结果，您可以使用字符串提示直接调用模型。

In [3]:
# Calling a single prompt

watsonx_llm.invoke("Who is man's best friend?")

"Man's best friend is his dog. Dogs are man's best friend because they are always there for you, they never judge you, and they love you unconditionally. Dogs are also great companions and can help reduce stress levels. "

In [4]:
# Calling multiple prompts

watsonx_llm.generate(
    [
        "The fastest dog in the world?",
        "Describe your chosen dog breed",
    ]
)

LLMResult(generations=[[Generation(text='The fastest dog in the world is the greyhound. Greyhounds can run up to 45 mph, which is about the same speed as a Usain Bolt.', generation_info={'finish_reason': 'eos_token'})], [Generation(text='The Labrador Retriever is a breed of retriever that was bred for hunting. They are a very smart breed and are very easy to train. They are also very loyal and will make great companions. ', generation_info={'finish_reason': 'eos_token'})]], llm_output={'token_usage': {'generated_token_count': 82, 'input_token_count': 13}, 'model_id': 'ibm/granite-13b-instruct-v2', 'deployment_id': None}, run=[RunInfo(run_id=UUID('750b8a0f-8846-456d-93d0-e039e95b1276')), RunInfo(run_id=UUID('aa4c2a1c-5b08-4fcf-87aa-50228de46db5'))], type='LLMResult')

## 流式传输模型输出

您可以流式传输模型输出。

In [5]:
for chunk in watsonx_llm.stream(
    "Describe your favorite breed of dog and why it is your favorite."
):
    print(chunk, end="")

My favorite breed of dog is a Labrador Retriever. They are my favorite breed because they are my favorite color, yellow. They are also very smart and easy to train. 

## 链式调用
创建 `PromptTemplate` 对象，它们将负责生成随机问题。

In [6]:
from langchain_core.prompts import PromptTemplate

template = "Generate a random question about {topic}: Question: "

prompt = PromptTemplate.from_template(template)

提供一个主题并运行链。

In [7]:
llm_chain = prompt | watsonx_llm

topic = "dog"

llm_chain.invoke(topic)

'What is the origin of the name "Pomeranian"?'

## API 参考

如需了解 `WatsonxLLM` 所有功能和配置的详细文档，请访问 [API 参考](https://python.langchain.com/api_reference/ibm/llms/langchain_ibm.llms.WatsonxLLM.html)。